# Space Titanic 데이터 분석 및 모델링

## 데이터 전처리 및 모델 학습

이 문서에서는 주어진 CSV 파일을 사용하여 데이터 전처리, 랜덤 포레스트 모델 훈련, 예측, 결과 저장 및 시각화를 수행합니다.

### 1. 라이브러리 불러오기

먼저 필요한 라이브러리를 불러옵니다.

In [ ]:
import pandas as pd

# 데이터 파일 경로 설정
train_file_path = 'train.csv'
test_file_path = 'test.csv'

# 데이터 로드: CSV 파일을 데이터프레임으로 읽어옵니다.
train_df = pd.read_csv(train_file_path)
test_df = pd.read_csv(test_file_path)

# 데이터의 기본 정보 확인: 컬럼명, 데이터 타입, 결측치 등 기본 정보를 출력합니다.
train_df.info()

# 데이터의 처음 몇 행을 출력하여 데이터의 구조를 확인합니다.
train_df.head()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# 상관 행렬 계산 및 히트맵 시각화
# 데이터 프레임의 상관 행렬을 계산합니다.
correlation_matrix = train_df.corr(numeric_only=True)

# 상관 행렬을 히트맵으로 시각화하여 각 변수 간의 상관 관계를 확인합니다.
plt.figure(figsize=(12, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', vmin=-1, vmax=1)
plt.title('Correlation Matrix')
plt.show()

plt.figure(figsize=(12, 8))  # 그래프 크기 설정
correlation_matrix = train_df.corr(numeric_only=True)  # 상관 행렬 계산

# 상관 관계를 내림차순으로 정렬
correlation_sorted = correlation_matrix[['Transported']].sort_values(by='Transported', ascending=False)

sns.heatmap(correlation_sorted, annot=True, cmap='coolwarm', vmin=-1, vmax=1)  # 히트맵 시각화
plt.title('Correlation of Features with Transported')  # 그래프 제목
plt.show()  # 그래프 출력


In [ ]:
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

### 2. 데이터 읽기 및 전처리 (Train 데이터)

2.1 CSV 파일을 읽어 데이터프레임 생성

In [ ]:
# 결측치 확인: 각 열에 대해 결측치가 있는지 확인합니다.
print("Train dataset missing values:")
print(train_df.isnull().sum())

print("\nTest dataset missing values:")
print(test_df.isnull().sum())



2.2 결측값 처리
결측값을 각 변수에 맞는 적절한 값으로 대체합니다.

각 정수형 값들에 대해선 평균으로,  
각 문자열에 대해선 Unkown 으로 표현합니다.

In [ ]:
train_df['Age'] = train_df['Age'].fillna(train_df['Age'].mean())
train_df['RoomService'] = train_df['RoomService'].fillna(train_df['RoomService'].mean())
train_df['FoodCourt'] = train_df['FoodCourt'].fillna(train_df['FoodCourt'].mean())
train_df['ShoppingMall'] = train_df['ShoppingMall'].fillna(train_df['ShoppingMall'].mean())
train_df['Spa'] = train_df['Spa'].fillna(train_df['Spa'].mean())
train_df['VRDeck'] = train_df['VRDeck'].fillna(train_df['VRDeck'].mean())

train_df['HomePlanet'] = train_df['HomePlanet'].fillna('Unknown')
train_df['Cabin'] = train_df['Cabin'].fillna('Unknown')
train_df['Destination'] = train_df['Destination'].fillna('Unknown')

# 결측치를 False로 처리합니다.
train_df['CryoSleep'] = train_df['CryoSleep'].fillna(False)
train_df['VIP'] = train_df['VIP'].fillna(False)

True 보단 False로 결측치를 처리했을 경우, 좋은 score가 나왔습니다.

2.3 데이터 타입 변환

In [ ]:
train_df['CryoSleep'] = train_df['CryoSleep'].astype(bool)
train_df['VIP'] = train_df['VIP'].astype(bool)

2.4 필요 없는 열 제거

In [ ]:
train_df.drop(columns=['Name', 'PassengerId'], inplace=True, errors='ignore')  # 분석에 필요 없는 열 제거

2.5 범주형 데이터 인코딩

In [ ]:
label_encoder = LabelEncoder()
train_df['HomePlanet'] = label_encoder.fit_transform(train_df['HomePlanet'])
train_df['Cabin'] = label_encoder.fit_transform(train_df['Cabin'])
train_df['Destination'] = label_encoder.fit_transform(train_df['Destination'])


2.6 정규화


In [ ]:
scaler = MinMaxScaler()
train_df[['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']] = scaler.fit_transform(train_df[['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']])


2.7 특징과 라벨 분리


In [ ]:
X_train = train_df.drop(columns=['Transported'])
y_train = train_df['Transported']


### 3. 랜덤 포레스트 모델 훈련

3.1 hyperparameter 후보 설정


In [ ]:
param_grid = {
    'n_estimators': [100, 300, 500, 1000],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt', 'log2']
}

# GridSearchCV 설정
grid_search = GridSearchCV(estimator=RandomForestClassifier(random_state=200),
                           param_grid=param_grid,
                           cv=5,
                           verbose=2)

In [ ]:
grid_search.fit(X_train, y_train)

In [ ]:
# 최적의 파라미터 출력
best_params = grid_search.best_params_
print("Best parameters found: ", best_params)
# 최적의 파라미터로 모델 설정
best_model = grid_search.best_estimator_

In [ ]:
# 최적의 파라미터로 모델 설정
best_model = grid_search.best_estimator_

### 4. 테스트 데이터 전처리 함수 정의

In [ ]:
def preprocess_test_data(test_df, label_encoder, scaler):
    test_df['Age'] = test_df['Age'].fillna(test_df['Age'].mean())
    test_df['RoomService'] = test_df['RoomService'].fillna(test_df['RoomService'].mean())
    test_df['FoodCourt'] = test_df['FoodCourt'].fillna(test_df['FoodCourt'].mean())
    test_df['ShoppingMall'] = test_df['ShoppingMall'].fillna(test_df['ShoppingMall'].mean())
    test_df['Spa'] = test_df['Spa'].fillna(test_df['Spa'].mean())
    test_df['VRDeck'] = test_df['VRDeck'].fillna(test_df['VRDeck'].mean())

    test_df['HomePlanet'] = test_df['HomePlanet'].fillna('Unknown')
    test_df['CryoSleep'] = test_df['CryoSleep'].fillna(False)
    test_df['Cabin'] = test_df['Cabin'].fillna('Unknown')
    test_df['Destination'] = test_df['Destination'].fillna('Unknown')
    test_df['VIP'] = test_df['VIP'].fillna(False)

    test_df['CryoSleep'] = test_df['CryoSleep'].astype(bool)
    test_df['VIP'] = test_df['VIP'].astype(bool)

    test_df.drop(columns=['Name'], inplace=True, errors='ignore')

    # train 데이터에 있는 범주형 값만 인코딩
    for column in ['HomePlanet', 'Cabin', 'Destination']:
        test_df[column] = test_df[column].apply(lambda x: x if x in label_encoder.classes_ else 'Unknown')

    test_df['HomePlanet'] = label_encoder.transform(test_df['HomePlanet'])
    test_df['Cabin'] = label_encoder.transform(test_df['Cabin'])
    test_df['Destination'] = label_encoder.transform(test_df['Destination'])

    test_df[['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']] = scaler.transform(test_df[['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']])

    return test_df


### 5. 테스트 데이터 전처리 및 예측

5.1 CSV 파일을 읽어 데이터프레임 생성 (Test 데이터)


In [ ]:
test_file_path = 'test.csv'  # 테스트 데이터 경로
test_df = pd.read_csv(test_file_path)  # 테스트 데이터를 데이터프레임으로 읽어옴


5.2 테스트 데이터 전처리


In [ ]:
test_df = preprocess_test_data(test_df, label_encoder, scaler)  # 정의한 전처리 함수 적용

5.3 특징 추출


In [ ]:
X_test = test_df.drop(columns=['Transported', 'PassengerId'], errors='ignore')  # 예측에 사용할 특징 추출


5.4 예측 및 결과 저장


In [ ]:
y_pred = model.predict(X_test)  # 모델을 사용하여 예측 수행

output = pd.DataFrame({'PassengerId': test_df['PassengerId'], 'Transported': y_pred})  # 결과 데이터프레임 생성
output.to_csv('prediction_results.csv', index=False)  # CSV 파일로 저장


### 6. 특성 중요도 시각화

In [ ]:
plt.figure(figsize=(12, 8))  # 그래프 크기 설정
sns.barplot(x=model.feature_importances_, y=X_train.columns)  # 특성 중요도를 막대 그래프로 시각화
plt.title('Feature Importances')  # 그래프 제목
plt.xlabel('Importance')  # x축 레이블
plt.ylabel('Feature')  # y축 레이블
plt.show()  # 그래프 출력
